In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List

# Конфигурация
API_BASE_URL = "http://localhost:8000"

print("=== ДЕМОНСТРАЦИЯ REST API ДЛЯ ПРЕДСКАЗАНИЯ ОТТОКА ===")

# 1. Проверка здоровья API
def check_health():
    """Проверка здоровья API"""
    try:
        response = requests.get(f"{API_BASE_URL}/health")
        if response.status_code == 200:
            health_data = response.json()
            print("✅ API здоров:")
            print(f"   Статус: {health_data['status']}")
            print(f"   Модель загружена: {health_data['model_loaded']}")
            print(f"   База данных подключена: {health_data['database_connected']}")
            return True
        else:
            print("❌ API не отвечает")
            return False
    except Exception as e:
        print(f"❌ Ошибка подключения к API: {e}")
        return False

# Проверяем здоровье
if not check_health():
    print("Запустите API командой: uvicorn api.main:app --reload")
    exit()

# 2. Получение информации о модели
def get_model_info():
    """Получение информации о модели"""
    response = requests.get(f"{API_BASE_URL}/model/info")
    if response.status_code == 200:
        model_info = response.json()
        print("\n📊 ИНФОРМАЦИЯ О МОДЕЛИ:")
        print(f"   Название: {model_info['model_name']}")
        print(f"   Версия: {model_info['model_version']}")
        print(f"   Тип: {model_info['model_type']}")
        print(f"   Количество признаков: {model_info['feature_count']}")
        print(f"   Дата обучения: {model_info['training_date']}")
        
        if model_info['performance']:
            print("   Метрики производительности:")
            for metric, value in model_info['performance'].items():
                print(f"     - {metric}: {value:.4f}")
        
        return model_info
    else:
        print("❌ Ошибка получения информации о модели")
        return None

model_info = get_model_info()

# 3. Пример данных для предсказания
sample_customer = {
    "Tenure": 12.5,
    "CityTier": 2.0,
    "WarehouseToHome": 8.0,
    "HourSpendOnApp": 3.2,
    "NumberOfDeviceRegistered": 3.0,
    "SatisfactionScore": 4.0,
    "NumberOfAddress": 5.0,
    "Complain": 0.0,
    "OrderAmountHikeFromlastYear": 15.0,
    "CouponUsed": 4.0,
    "OrderCount": 25.0,
    "DaySinceLastOrder": 3.0,
    "CashbackAmount": 250.0,
    "AvgOrdersPerMonth": 2.0,
    "AvgCashbackPerOrder": 10.0,
    "EngagementScore": 80.0,
    "SatisfactionComplainRatio": 4.0
}

print(f"\n🎯 ПРИМЕР ДАННЫХ КЛИЕНТА:")
for feature, value in sample_customer.items():
    print(f"   {feature}: {value}")

# 4. Одиночное предсказание
def predict_single(customer_data: Dict, customer_id: int = None, threshold: float = 0.5):
    """Одиночное предсказание"""
    params = {"threshold": threshold}
    if customer_id:
        params["customer_id"] = customer_id
    
    response = requests.post(
        f"{API_BASE_URL}/predict",
        json=customer_data,
        params=params
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Ошибка предсказания: {response.text}")
        return None

print("\n🔮 ОДИНОЧНОЕ ПРЕДСКАЗАНИЕ:")
prediction = predict_single(sample_customer, customer_id=50001, threshold=0.5)

if prediction:
    print(f"   ID клиента: {prediction['customer_id']}")
    print(f"   Вероятность оттока: {prediction['churn_probability']:.4f}")
    print(f"   Предсказание: {'ОТТОК' if prediction['prediction'] else 'НЕТ ОТТОКА'}")
    print(f"   Версия модели: {prediction['model_version']}")

# 5. Пакетное предсказание
def predict_batch(customers_data: List[Dict], customer_ids: List[int] = None, threshold: float = 0.5):
    """Пакетное предсказание"""
    request_data = {"customers": customers_data}
    if customer_ids:
        request_data["customer_ids"] = customer_ids
    
    response = requests.post(
        f"{API_BASE_URL}/predict/batch",
        json=request_data,
        params={"threshold": threshold}
    )
    
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Ошибка пакетного предсказания: {response.text}")
        return None

# Создаем несколько тестовых клиентов
batch_customers = [
    {
        "Tenure": 24.0, "CityTier": 1.0, "WarehouseToHome": 5.0,
        "HourSpendOnApp": 2.0, "NumberOfDeviceRegistered": 2.0,
        "SatisfactionScore": 5.0, "NumberOfAddress": 3.0, "Complain": 0.0,
        "OrderAmountHikeFromlastYear": 10.0, "CouponUsed": 8.0,
        "OrderCount": 40.0, "DaySinceLastOrder": 1.0, "CashbackAmount": 400.0
    },
    {
        "Tenure": 3.0, "CityTier": 3.0, "WarehouseToHome": 20.0,
        "HourSpendOnApp": 1.0, "NumberOfDeviceRegistered": 1.0,
        "SatisfactionScore": 2.0, "NumberOfAddress": 10.0, "Complain": 1.0,
        "OrderAmountHikeFromlastYear": 25.0, "CouponUsed": 1.0,
        "OrderCount": 5.0, "DaySinceLastOrder": 15.0, "CashbackAmount": 50.0
    },
    sample_customer  # Добавляем нашего первого клиента
]

batch_ids = [50002, 50003, 50004]

print("\n📦 ПАКЕТНОЕ ПРЕДСКАЗАНИЕ:")
batch_prediction = predict_batch(batch_customers, batch_ids, threshold=0.5)

if batch_prediction:
    print(f"   Всего клиентов: {batch_prediction['total_customers']}")
    print(f"   Процент оттока в батче: {batch_prediction['churn_rate']:.2%}")
    
    print("\n   Результаты предсказаний:")
    for i, pred in enumerate(batch_prediction['predictions']):
        status = "ОТТОК" if pred['prediction'] else "НЕТ ОТТОКА"
        print(f"     Клиент {pred['customer_id']}: {pred['churn_probability']:.4f} - {status}")

# 6. Анализ влияния threshold
print("\n📈 АНАЛИЗ ВЛИЯНИЯ THRESHOLD:")

thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
results = []

for threshold in thresholds:
    pred = predict_single(sample_customer, threshold=threshold)
    if pred:
        results.append({
            'threshold': threshold,
            'probability': pred['churn_probability'],
            'prediction': pred['prediction']
        })

if results:
    results_df = pd.DataFrame(results)
    print(results_df.to_string(index=False))

# 7. Визуализация результатов
if batch_prediction:
    # Создаем DataFrame с результатами
    pred_df = pd.DataFrame([p.dict() for p in batch_prediction['predictions']])
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Распределение вероятностей
    axes[0].hist(pred_df['churn_probability'], bins=10, alpha=0.7, color='skyblue', edgecolor='black')
    axes[0].axvline(x=0.5, color='red', linestyle='--', label='Threshold = 0.5')
    axes[0].set_xlabel('Вероятность оттока')
    axes[0].set_ylabel('Количество клиентов')
    axes[0].set_title('Распределение вероятностей оттока')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Статус оттока
    churn_counts = pred_df['prediction'].value_counts()
    axes[1].pie(churn_counts.values, labels=['Нет оттока', 'Отток'], 
               autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
    axes[1].set_title('Распределение предсказаний оттока')
    
    plt.tight_layout()
    plt.show()

# 8. Получение списка признаков
def get_feature_list():
    """Получение списка признаков"""
    response = requests.get(f"{API_BASE_URL}/features/list")
    if response.status_code == 200:
        return response.json()
    else:
        print("❌ Ошибка получения списка признаков")
        return None

print("\n📋 СПИСОК ПРИЗНАКОВ МОДЕЛИ:")
features_info = get_feature_list()
if features_info:
    print(f"   Всего признаков: {features_info['total_features']}")
    print("   Первые 10 признаков:")
    for feature in features_info['features'][:10]:
        print(f"     - {feature}")

# 9. Тестирование валидации
print("\n🧪 ТЕСТИРОВАНИЕ ВАЛИДАЦИИ:")

# Невалидные данные - SatisfactionScore вне диапазона
invalid_customer = sample_customer.copy()
invalid_customer["SatisfactionScore"] = 10.0  # Невалидное значение

response = requests.post(f"{API_BASE_URL}/predict", json=invalid_customer)
if response.status_code == 400:
    print("✅ Валидация работает: невалидный SatisfactionScore отклонен")
else:
    print("❌ Валидация не сработала")

# Неполные данные
incomplete_customer = {"Tenure": 12.5}  # Только одно поле

response = requests.post(f"{API_BASE_URL}/predict", json=incomplete_customer)
if response.status_code == 422:
    print("✅ Валидация работает: неполные данные отклонены")
else:
    print("❌ Валидация не сработала")

print("\n🎉 ДЕМОНСТРАЦИЯ ЗАВЕРШЕНА!")
print("\nДоступные эндпоинты:")
print("  GET  /health              - Проверка здоровья API")
print("  GET  /model/info          - Информация о модели")
print("  POST /predict             - Одиночное предсказание")
print("  POST /predict/batch       - Пакетное предсказание")
print("  GET  /features/list       - Список признаков")
print("  GET  /customer/{id}/predict - Предсказание по ID клиента")
print("  GET  /customer/{id}/history - История предсказаний")

print(f"\nДокументация API: {API_BASE_URL}/docs")